In [1]:
from IPython.display import HTML
import random

def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Toggle show/hide'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)

hide_toggle()

***Подгружаем необходимые инструменты:***

In [2]:
import warnings

warnings.filterwarnings('ignore')

hide_toggle()

In [3]:
import numpy as np
import pandas as pd
import gzip
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from collections import Counter
import string

import nltk
from nltk import *
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
import re

from gensim import models, corpora

#import polyglot
#from polyglot.text import Text as T
nltk.download()
hide_toggle()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


***Загружаем текстовые данные:***

In [4]:
path = 'qa_Appliances.json.gz'


def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('qa_Appliances.json.gz')

#data_answers = df['answer']
data_answers = df['answer'].apply(lambda x: x.lower())
#data_answers = str(data_answers)
data_answers

hide_toggle()

In [5]:
data_answers.head(5)

0     i replaced my old one with this without a hitch.
1    this may help insinkerator model badger-1: bad...
2    plumbing connections will vary with different ...
3    it does not come with a power cord. it does co...
4    check if you dropped something inside.usually ...
Name: answer, dtype: object

***Функции по очистке данных (освобождение от цифр и знаков пунктуации):***

In [6]:
#cleaning and processing

#cleaning from punctuation
def replace_punctuation(x):  # From S.Lott's solution
    for c in string.punctuation:
        x=x.replace(c,"")
    return x

#removing numbers
def clean_numbers(x):
    x = str(x)
    re.sub(r'\d+', '', x)
    return x

hide_toggle()

***Вычисляем частотность слов (N самых употребляемых слов):***

In [7]:

#top N words     всего текста
def top_N_words(N):
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf.fit(data_answers)
    X = tfidf.transform(data_answers)
    idx = np.ravel(X.sum(axis=0).argsort(axis=1))[::-1][:N]
    top_n_words = np.array(tfidf.get_feature_names())[idx].tolist()
    return top_n_words


hide_toggle()

In [8]:
top_N_words(10)

['yes',
 'model',
 'fit',
 'don',
 'sure',
 'does',
 'know',
 'just',
 'work',
 'filter']

***Вычисление эмоциональной окрашенности текста:***

In [9]:
#sentiment analysis
#for english version we'll use vader sentiment analyzer

def emotions(data_answers):
    data_answers = str(data_answers)
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    sid = SentimentIntensityAnalyzer()
    sentences = tokenize.sent_tokenize(data_answers)
    scores = dict([('pos', 0), ('neu', 0), ('neg', 0), ('compound', 0)])
    for sentence in sentences:
        ss = sid.polarity_scores(sentence)
        for k in sorted(ss):
            scores[k] += ss[k]

    return scores 


hide_toggle()

In [10]:
emotions(data_answers)

{'compound': 6.4398,
 'neg': 0.21400000000000002,
 'neu': 20.912999999999997,
 'pos': 2.8729999999999998}

***Вычленение главных тем:***      (тест)

***вариант-1***

In [11]:
#LDA gensim version-2
import re
from gensim import models, corpora
from nltk import word_tokenize
from nltk.corpus import stopwords
 
NUM_TOPICS = 10
STOPWORDS = stopwords.words('english')
texts = str(data_answers)
#texts = text.split()
    
def clean_text(texts):
    tokenized_text = word_tokenize(texts.lower())
    cleaned_text = [t for t in tokenized_text if t not in STOPWORDS and re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', t)]
    return cleaned_text
 
# For gensim we need to tokenize the data and filter out stopwords
tokenized_data = []
for text in texts:
    tokenized_data.append(clean_text(texts))
 
 
# Build a Dictionary - association word to numeric id
dictionary = corpora.Dictionary(tokenized_data)
 
# Transform the collection of texts to a numerical form
corpus = [dictionary.doc2bow(text) for text in tokenized_data]
 
# Have a look at how the 20th document looks like: [(word_id, count), ...]
print(corpus[20])
# [(12, 3), (14, 1), (21, 1), (25, 5), (30, 2), (31, 5), (33, 1), (42, 1), (43, 2),  ...
 
# Build the LDA model
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)
 
# Build the LSI model
lsi_model = models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

print("LDA Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lda_model.print_topic(idx, 10))
 
print("=" * 20)
 
print("LSI Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lsi_model.print_topic(idx, 10))
 
print("=" * 20)

hide_toggle()

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 4), (16, 2), (17, 2), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2), (42, 1), (43, 1), (44, 1), (45, 3), (46, 1), (47, 1), (48, 1), (49, 1), (50, 7), (51, 1), (52, 1), (53, 2), (54, 1), (55, 1), (56, 3), (57, 1), (58, 1), (59, 1), (60, 2), (61, 1), (62, 4), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 2), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 2), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 2), (89, 1), (90, 2), (91, 1), (92, 1), (93, 1), (94, 3), (95, 1), (96, 1), (97, 1), (98, 3), (99, 2), (100, 3), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 2), (109, 1), (110, 1),

***вариант-2***

In [12]:
#LDA scikit-learn version-2

from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
 
NUM_TOPICS = 10
 
vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(data_answers)
 
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
print(lda_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Non-Negative Matrix Factorization Model
nmf_model = NMF(n_components=NUM_TOPICS)
nmf_Z = nmf_model.fit_transform(data_vectorized)
print(nmf_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Latent Semantic Indexing Model
lsi_model = TruncatedSVD(n_components=NUM_TOPICS)
lsi_Z = lsi_model.fit_transform(data_vectorized)
print(lsi_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 

#Let's see how the first document in the corpus looks like in different topic spaces
print(lda_Z[0])
print(nmf_Z[0])
print(lsi_Z[0])


def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])

print("LDA Model:")
print_topics(lda_model, vectorizer)
print("=" * 20)
 
print("NMF Model:")
print_topics(nmf_model, vectorizer)
print("=" * 20)
 
print("LSI Model:")
print_topics(lsi_model, vectorizer)
print("=" * 20)

hide_toggle()

(9011, 10)
(9011, 10)
(9011, 10)
[0.03334315 0.03333636 0.03334    0.03333682 0.03333333 0.03333421
 0.03333333 0.69997614 0.03333333 0.03333333]
[0.         0.         0.00236963 0.00025976 0.00204395 0.01965884
 0.         0.         0.         0.        ]
[ 0.0866132   0.05048539 -0.01218395 -0.03645353 -0.01797298 -0.05034853
 -0.07541779  0.01931707 -0.07592397 -0.05868871]
LDA Model:
Topic 0:
[('filter', 959.2335677947433), ('filters', 265.9766588007707), ('sorry', 234.56882576267716), ('yes', 206.52494106590112), ('refrigerator', 200.98362180267026), ('box', 154.11648738873595), ('fridge', 141.2158548155627), ('installed', 137.63660195150416), ('water', 120.6836898885183), ('according', 110.69609373315932)]
Topic 1:
[('model', 937.8121782970042), ('number', 746.2867341590437), ('parts', 361.53023230053685), ('whirlpool', 351.38529805691326), ('sure', 308.44228387442524), ('work', 265.81319820145467), ('fit', 257.40033786823807), ('amazon', 233.16327527262072), ('just', 218.05719

[('com', 0.32604772171615126), ('http', 0.28180278714905477), ('www', 0.2750172909013333), ('amazon', 0.22242049129191155), ('washer', 0.20386937294950477), ('door', 0.14902844011180194), ('need', 0.13723780668979946), ('ice', 0.12101860186244012), ('dryer', 0.12040485624499836), ('filters', 0.09144455280478857)]


***Визуализация выделенных тем:***

In [13]:
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
5       32.696026  167.590805       1        1  13.160854
1       93.067963  120.921867       2        1  12.738256
4        1.407326   52.538681       3        1  11.764405
3       60.789505   59.461735       4        1  11.177940
9      120.708359   49.930298       5        1  10.589492
7      -36.059814  124.468208       6        1   9.943992
2      -11.203334   -7.950607       7        1   8.335786
8      -60.179214   57.306091       8        1   8.256193
6       25.153149  107.405769       9        1   7.158305
0       59.992702   -4.010499      10        1   6.874777, topic_info=     Category         Freq           Term        Total  loglift  logprob
893   Default   917.000000         filter   917.000000  30.0000  30.0000
2678  Default  1602.000000            yes  1602.000000  29.0000  29.0000
1528  Default  1099.000000          model  1099.000000  28.0000  28.0000
2585  Default  1076.000000          water  1076.000000  27.0000  27.0000
1597  Default   721.000000         number   721.000000  26.0000  26.0000
1174  Default   515.000000            ice   515.000000  25.0000  25.0000
443   Default   389.000000            com   389.000000  24.0000  24.0000
2577  Default   456.000000         washer   456.000000  23.0000  23.0000
1275  Default   884.000000           just   884.000000  22.0000  22.0000
2647  Default   656.000000           work   656.000000  21.0000  21.0000
1300  Default   634.000000           know   634.000000  20.0000  20.0000
739   Default   397.000000          dryer   397.000000  19.0000  19.0000
1957  Default   306.000000    replacement   306.000000  18.0000  18.0000
1419  Default   362.000000        machine   362.000000  17.0000  17.0000
1195  Default   336.000000         inches   336.000000  16.0000  16.0000
907   Default   791.000000            fit   791.000000  15.0000  15.0000
80    Default   417.000000         amazon   417.000000  14.0000  14.0000
1025  Default   518.000000           good   518.000000  13.0000  13.0000
2341  Default   775.000000           sure   775.000000  12.0000  12.0000
245   Default   280.000000         bought   280.000000  11.0000  11.0000
705   Default   800.000000            don   800.000000  10.0000  10.0000
2669  Default   260.000000            www   260.000000   9.0000   9.0000
1649  Default   275.000000           oven   275.000000   8.0000   8.0000
1039  Default   410.000000          great   410.000000   7.0000   7.0000
896   Default   253.000000        filters   253.000000   6.0000   6.0000
1688  Default   423.000000          parts   423.000000   5.0000   5.0000
1161  Default   253.000000           http   253.000000   4.0000   4.0000
2612  Default   372.000000      whirlpool   372.000000   3.0000   3.0000
1922  Default   312.000000   refrigerator   312.000000   2.0000   2.0000
2213  Default   223.000000          sorry   223.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
512   Topic10    29.308086   contaminants    30.183656   2.6479  -5.5021
1923  Topic10    29.063535  refrigerators    29.938374   2.6477  -5.5105
366   Topic10    27.850560       charcoal    28.725330   2.6464  -5.5531
2513  Topic10    27.556087            usa    28.430944   2.6461  -5.5637
367   Topic10    24.654556         charge    25.529424   2.6424  -5.6750
1423  Topic10    24.453252       magnetic    25.328190   2.6422  -5.6832
1443  Topic10    24.293764   manufactured    25.168599   2.6419  -5.6897
510   Topic10    22.177592      container    23.052565   2.6386  -5.7809
1693  Topic10    21.872735           past    22.747638   2.6381  -5.7947
385   Topic10    21.506088       chlorine    22.380901   2.6374  -5.8116
1853  Topic10    83.153590        quality    87.947076   2.6213  -4.4593
1922  Topic10   191.082409   refrigerator   312.200510   2.1864  -3.6272
881   Topic10    46.341126           feel    57.4